### Import important libraries

In [112]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from typing import Dict, List, Optional
import time
import asyncio
import aiohttp
from web3 import Web3


### Load environment variable, connect to ethereum and API setup 

In [113]:
# Load environment variables
load_dotenv()

# API Keys
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")

WEB3_PROVIDER_URL = os.getenv("WEB3_PROVIDER_URL")
w3 = Web3(Web3.HTTPProvider(WEB3_PROVIDER_URL))

# Check if the connection is successful
if w3.is_connected():
    print("Connected to Ethereum network.")
else:
    print("Connection failed.")

def check_api_keys() -> Dict[str, bool]:
    """Check if API keys are loaded"""
    return {
        "etherscan": bool(ETHERSCAN_API_KEY),
        "coingecko": bool(COINGECKO_API_KEY)
    }

Connection failed.


In [114]:
check_api_keys()

{'etherscan': True, 'coingecko': True}

### Get eth balance

In [115]:
# The simplified ABI for ERC-20 tokens (this is required to interact with the contract)
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function",
    }
]

In [116]:
def get_eth_balance(wallet_address: str, w3: Web3) -> float:
    """Get the ETH balance of a wallet using web3.py."""
    try:
        balance_wei = w3.eth.get_balance(w3.to_checksum_address(wallet_address))
        balance_eth = w3.from_wei(balance_wei, 'ether')
        return float(balance_eth)
    except Exception as e:
        print(f"Error getting ETH balance: {e}")
        return 0


In [117]:
balance = get_eth_balance("0x742d35Cc6634C0532925a3b844Bc454e4438f44e", w3)
print(balance)

Error getting ETH balance: HTTPSConnectionPool(host='eth-mainnet.g.alchemy.com', port=443): Max retries exceeded with url: /v2/OrFcVw7gBiw8VsPO9cLljK1mzs9NqFIz (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000025C7DDD3550>: Failed to resolve 'eth-mainnet.g.alchemy.com' ([Errno 11001] getaddrinfo failed)"))
0


### Get ETH price

In [118]:
async def get_eth_price(session) -> float:
    """Get current ETH price from CoinGecko"""
    try:
        url = "https://api.coingecko.com/api/v3/simple/price"
        params = {
            "ids": "ethereum",
            "vs_currencies": "usd",
            "include_24hr_change": "true"
        }
        
        async with session.get(url, params=params, timeout=10) as response:
            response.raise_for_status()
            data = await response.json()
            
            price_data = data.get("ethereum", {})
            return {
                "price": price_data.get("usd", 0),
                "change_24h": price_data.get("usd_24h_change", 0)
            }
        
    except Exception as e:
        print(f"Error getting ETH price: {e}")
        return {"price": 0, "change_24h": 0}

In [119]:
async with aiohttp.ClientSession() as session:
    price = await get_eth_price(session)
    print(price)

Error getting ETH price: Cannot connect to host api.coingecko.com:443 ssl:default [getaddrinfo failed]
{'price': 0, 'change_24h': 0}


### token Decimal and balance

In [120]:
TOKEN_DECIMALS_CACHE = {}
async def get_token_decimals(contract_address: str, session) -> int:
    """Get token decimals from cache or Etherscan API."""
    if contract_address in TOKEN_DECIMALS_CACHE:
        return TOKEN_DECIMALS_CACHE[contract_address]
        
    try:
        url = "https://api.etherscan.io/api"
        params = {
            "module": "token",
            "action": "tokeninfo",
            "contractaddress": contract_address,
            "apikey": ETHERSCAN_API_KEY
        }
        
        async with session.get(url, params=params, timeout=10) as response:
            response.raise_for_status()
            data = await response.json()
            
            if data["status"] == "1":
                decimals = int(data["result"][0]["decimals"])
                TOKEN_DECIMALS_CACHE[contract_address] = decimals
                return decimals
        
    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Error getting token decimals for {contract_address}: {e}")
        
    # Default to 18 on API failure or if not found
    TOKEN_DECIMALS_CACHE[contract_address] = 18
    return 18

def get_token_balance_dynamic(wallet_address: str, contract_address: str, w3: Web3) -> Dict:
    """
    Get the balance, symbol, and decimals of a specific ERC-20 token.
    Note: The name is retained for consistency, but it now directly queries the contract.
    """
    try:
        token_contract = w3.eth.contract(
            address=w3.to_checksum_address(contract_address),
            abi=ERC20_ABI
        )
        balance_wei = token_contract.functions.balanceOf(
            w3.to_checksum_address(wallet_address)
        ).call()
        decimals = token_contract.functions.decimals().call()
        symbol = token_contract.functions.symbol().call()
        
        balance = balance_wei / (10 ** decimals)
        
        return {
            "address": contract_address,
            "symbol": symbol,
            "balance": balance
        }
    except Exception as e:
        print(f"Error getting token balance for {contract_address}: {e}")
        return {
            "address": contract_address,
            "symbol": "Unknown",
            "balance": 0
        }

In [121]:
wallet_address = "0x742d35Cc6634C0532925a3b844Bc454e4438f44e"
contract_address = "0x6982508145454ce325ddbe47a25d4ec3d2311933"

token_balance = get_token_balance_dynamic(wallet_address, contract_address, w3)
print(token_balance)

Error getting token balance for 0x6982508145454ce325ddbe47a25d4ec3d2311933: HTTPSConnectionPool(host='eth-mainnet.g.alchemy.com', port=443): Max retries exceeded with url: /v2/OrFcVw7gBiw8VsPO9cLljK1mzs9NqFIz (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000025C7DF4ED70>: Failed to resolve 'eth-mainnet.g.alchemy.com' ([Errno 11001] getaddrinfo failed)"))
{'address': '0x6982508145454ce325ddbe47a25d4ec3d2311933', 'symbol': 'Unknown', 'balance': 0}


### Get erc-20 holdings

In [122]:
# async def get_erc20_tokens(wallet_address: str, session) -> List[Dict]:
#     """
#     Get a list of all ERC-20 tokens with a non-zero balance for a wallet.
    
#     Args:
#         wallet_address (str): The Ethereum wallet address.
#         session (aiohttp.ClientSession): The async session object.
    
#     Returns:
#         List[Dict]: A list of dictionaries, where each dict represents a token with
#                     its contract address, symbol, and decimals.
#     """
#     try:
#         # Step 1: Get all token transfer events to find unique tokens
#         url = "https://api.etherscan.io/api"
#         params = {
#             "module": "account",
#             "action": "tokentx",
#             "address": wallet_address,
#             "apikey": ETHERSCAN_API_KEY
#         }
        
#         async with session.get(url, params=params, timeout=20) as response:
#             response.raise_for_status()
#             tx_data = await response.json()
            
#             if tx_data["status"] != "1":
#                 return []
            
#             # Extract unique contract addresses from the transaction history
#             unique_tokens = {
#                 tx["contractAddress"]: {
#                     "address": tx["contractAddress"],
#                     "symbol": tx["tokenSymbol"]
#                 } for tx in tx_data["result"]
#             }
            
#             # Step 2: Check the balance for each unique token concurrently
#             balance_tasks = [
#                 get_token_balance_dynamic(wallet_address, token_info["address"], session)
#                 for token_info in unique_tokens.values()
#             ]
#             balances = await asyncio.gather(*balance_tasks)
            
#             # Combine token info with balance and filter out zero balances
#             held_tokens = []
#             for i, token_info in enumerate(unique_tokens.values()):
#                 if balances[i] > 0:
#                     held_tokens.append({
#                         "address": token_info["address"],
#                         "symbol": token_info["symbol"],
#                         "balance": balances[i]
#                     })
            
#             return held_tokens
            
#     except Exception as e:
#         print(f"Error getting ERC-20 tokens: {e}")
#         return []

In [123]:
# async with aiohttp.ClientSession() as session:
#     tokens = await get_erc20_tokens("0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97", session)  # Example wallet address
#     print(tokens)

### Get prices of tokens

#### Get list of tokens supported by coingecko

In [124]:
async def get_coingecko_token_list(session: aiohttp.ClientSession) -> set:
    """
    Fetches a set of all token IDs known to CoinGecko.
    """
    try:
        url = "https://api.coingecko.com/api/v3/coins/list"
        async with session.get(url, timeout=20) as response:
            response.raise_for_status()
            data = await response.json()
            # We're interested in the ID for the token, not the contract address for this endpoint
            return {item['id'] for item in data}
    except Exception as e:
        print(f"Error getting CoinGecko token list: {e}")
        return set()

In [125]:
async def get_multiple_token_prices(token_addresses: List[str], session: aiohttp.ClientSession) -> Dict:
    """
    Fetches the price for a list of ERC-20 tokens, handling errors for unknown tokens individually.
    """
    prices = {}
    
    # Create a list of tasks for each token
    price_tasks = []
    for addr in token_addresses:
        # Create a task for each individual token price request
        task = asyncio.create_task(get_single_token_price(addr, session))
        price_tasks.append(task)
        
    # Run all tasks concurrently
    results = await asyncio.gather(*price_tasks, return_exceptions=True)
    
    # Process the results
    for addr, result in zip(token_addresses, results):
        if not isinstance(result, Exception):
            # If the call was successful, add the price data
            prices.update(result)
        else:
            # If there was an error, print a message but continue
            print(f"Failed to get price for {addr}. Error: {result}")
            
    return prices


async def get_single_token_price(token_address: str, session: aiohttp.ClientSession) -> Dict:
    """
    Fetches the price for a single token, returning an empty dict on failure.
    """
    try:
        url = "https://api.coingecko.com/api/v3/simple/token_price/ethereum"
        params = {
            "contract_addresses": token_address.lower(),
            "vs_currencies": "usd",
            "include_24hr_change": "true"
        }
        
        async with session.get(url, params=params, timeout=10) as response:
            response.raise_for_status()
            data = await response.json()
            return data
    except Exception as e:
        # The exception is returned and handled by the caller
        raise e

In [126]:
async def get_single_token_price(contract_address: str, session) -> Dict:
    """Get price for a single token from CoinGecko API."""
    try:
        url = "https://api.coingecko.com/api/v3/simple/token_price/ethereum"
        params = {
            "contract_addresses": contract_address.lower(),
            "vs_currencies": "usd",
            "include_24hr_change": "true"
        }
        
        async with session.get(url, params=params, timeout=10) as response:
            response.raise_for_status()
            data = await response.json()
            # The API returns a dictionary with the contract address as the key.
            return data.get(contract_address.lower(), {})
            
    except Exception as e:
        print(f"Error getting token price for {contract_address}: {e}")
        return {}

async def get_portfolio_data_with_workaround(token_addresses: List[str]):
    async with aiohttp.ClientSession() as session:
        # Create a task for each token
        tasks = [get_single_token_price(addr, session) for addr in token_addresses]
        
        # Run all tasks concurrently and get the results
        results = await asyncio.gather(*tasks)
        
        # Combine the results into a single dictionary
        combined_prices = {}
        for i, addr in enumerate(token_addresses):
            combined_prices[addr.lower()] = results[i]
            
        return combined_prices

In [127]:
async with aiohttp.ClientSession() as session:
    token_addresses = [
        "0x6b175474e89094c44da98b954eedeac495271d0f",  # DAI
        "0x6982508145454ce325ddbe47a25d4ec3d2311933"   # PEPE
    ]
    prices = await get_portfolio_data_with_workaround(token_addresses)
    print(prices)

Error getting token price for 0x6982508145454ce325ddbe47a25d4ec3d2311933: Cannot connect to host api.coingecko.com:443 ssl:default [getaddrinfo failed]
Error getting token price for 0x6b175474e89094c44da98b954eedeac495271d0f: Cannot connect to host api.coingecko.com:443 ssl:default [getaddrinfo failed]
{'0x6b175474e89094c44da98b954eedeac495271d0f': {}, '0x6982508145454ce325ddbe47a25d4ec3d2311933': {}}


### Get list of tokens held

In [128]:
async def get_held_tokens_alchemy(wallet_address: str, session: aiohttp.ClientSession) -> List[Dict]:
    """
    Fetches all ERC-20 tokens held by a wallet using the Alchemy API.
    
    Args:
        wallet_address (str): The Ethereum wallet address.
        session (aiohttp.ClientSession): The async session object.

    Returns:
        List[Dict]: A list of dictionaries, where each dict represents a token with
                    its contract address.
    """
    # Replace with your actual Alchemy API URL
    ALCHEMY_API_URL = WEB3_PROVIDER_URL
    headers = {"accept": "application/json", "content-type": "application/json"}
    
    payload = {
        "id": 1,
        "jsonrpc": "2.0",
        "method": "alchemy_getTokenBalances",
        "params": [
            wallet_address,
            "erc20"
        ]
    }
    
    try:
        async with session.post(ALCHEMY_API_URL, headers=headers, json=payload, timeout=20) as response:
            response.raise_for_status()
            data = await response.json()
            
            held_tokens = []
            if "result" in data and "tokenBalances" in data["result"]:
                for token_info in data["result"]["tokenBalances"]:
                    # Alchemy's API returns tokens with a zero balance too; we filter them.
                    # The balance is in hexadecimal format, so we check for "0x0".
                    if token_info["tokenBalance"] != "0x0":
                        # The contract address is correctly extracted here
                        held_tokens.append({"address": token_info["contractAddress"]})

            return held_tokens
    
    except Exception as e:
        print(f"Error getting held tokens from Alchemy: {e}")
        return []

In [129]:
async with aiohttp.ClientSession() as session:
    tokensLists = await get_held_tokens_alchemy("0x742d35Cc6634C0532925a3b844Bc454e4438f44e", session)
    print(tokensLists)
    


Error getting held tokens from Alchemy: Cannot connect to host eth-mainnet.g.alchemy.com:443 ssl:default [getaddrinfo failed]
[]


In [130]:
async with aiohttp.ClientSession() as session:
    held_tokens_list = await get_held_tokens_alchemy(wallet_address, session)
df_held_tokens_list = pd.DataFrame(tokensLists)

df_held_tokens_list.head()  # Display the first few rows of the DataFrame

Error getting held tokens from Alchemy: Cannot connect to host eth-mainnet.g.alchemy.com:443 ssl:default [getaddrinfo failed]


""


In [131]:
df = pd.DataFrame(tokensLists)
df.head()  # Display the first few rows of the DataFrame

""


### Get wallet portifolio data

In [132]:
async def get_portfolio_data(wallet_address: str) -> Dict:
    async with aiohttp.ClientSession() as session:
        # Step 1: Get ETH balance, ETH price, and held tokens concurrently
        eth_balance_task = asyncio.to_thread(get_eth_balance, wallet_address, w3)
        eth_price_task = asyncio.create_task(get_eth_price(session))
        held_tokens_task = asyncio.create_task(get_held_tokens_alchemy(wallet_address, session))

        eth_balance, eth_price_data, held_tokens_list = await asyncio.gather(
            eth_balance_task, eth_price_task, held_tokens_task
        )
        
        portfolio = {
            "wallet_address": wallet_address,
            "eth_balance": eth_balance,
            "eth_price": eth_price_data["price"],
            "eth_value": eth_balance * eth_price_data["price"],
            "tokens": [],
            "total_value": eth_balance * eth_price_data["price"],
        }
        
        if held_tokens_list:
            token_addresses = [token["address"] for token in held_tokens_list]
            
            # Step 2: Get all token prices concurrently
            token_prices_task = asyncio.create_task(get_multiple_token_prices(token_addresses, session))
            token_prices = await token_prices_task
            
            # Step 3: Get all token balances concurrently
            token_balance_tasks = [
                asyncio.to_thread(get_token_balance_dynamic, wallet_address, addr, w3)
                for addr in token_addresses
            ]
            token_balances = await asyncio.gather(*token_balance_tasks)
            
            for token_info in token_balances:
                if token_info["balance"] > 0:
                    addr = token_info["address"].lower()
                    price_data = token_prices.get(addr, {})
                    token_info["price"] = price_data.get("usd", 0)
                    token_info["value"] = token_info["balance"] * token_info["price"]
                    portfolio["tokens"].append(token_info)
                    portfolio["total_value"] += token_info["value"]

        return portfolio

In [135]:
async def main():
    wallet_address_to_track = "0xd40E9C8cE75615C91e6D3c56f9A16C65A6BD3b35" # A test wallet
        
    portfolio = await get_portfolio_data(wallet_address_to_track)
    
    print(f"--- Portfolio Data for {portfolio['wallet_address']} ---")
    print(f"ETH Balance: {portfolio['eth_balance']:.4f} ETH")
    print(f"Total Portfolio Value: ${portfolio['total_value']:.2f}")
    print("\nTokens:")
    for token in portfolio['tokens']:
        print(f"  - {token['symbol']}: {token['balance']:.4f} (${token['value']:.2f})")
    print("---------------------------------")

In [136]:
# asyncio.run(main())

await main()

Error getting token price for 0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce: 429, message='Too Many Requests', url='https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses=0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce&vs_currencies=usd&include_24hr_change=true'
Error getting token price for 0xdac17f958d2ee523a2206206994597c13d831ec7: 429, message='Too Many Requests', url='https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses=0xdac17f958d2ee523a2206206994597c13d831ec7&vs_currencies=usd&include_24hr_change=true'
Error getting token price for 0x514910771af9ca656af840dff83e8264ecf986ca: 429, message='Too Many Requests', url='https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses=0x514910771af9ca656af840dff83e8264ecf986ca&vs_currencies=usd&include_24hr_change=true'
--- Portfolio Data for 0xd40E9C8cE75615C91e6D3c56f9A16C65A6BD3b35 ---
ETH Balance: 5.8667 ETH
Total Portfolio Value: $24580.07

Tokens:
  - MNT: 898.7068 ($0.00